In [10]:
# IMPORTS

import keras.layers as layers
import matplotlib.pyplot as plt
from keras import Model
from numpy import argmax
from keras.optimizers import SGD, Adam
from pandas import read_csv
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc
from keras.models import Sequential
from keras.utils import to_categorical
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator # In Colab, change to keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
import gradio as gr
import random
import shutil

In [11]:
# CREATE GRADIO INTERFACE

IMAGE_SIZE=(224,224) # images will be resized to this
def build_and_compile_cnn():
    """
    Build and compile a Convolutional Neural Network model.

    The CNN consists of:
    - Three convolutional layers with ReLU activation and max pooling layers.
    - A flattening layer to convert 2D matrices into a 1D vector.
    - A dense (fully connected) layer with ReLU activation.
    - A dense output layer with sigmoid activation for binary classification.

    The model is compiled using the RMSProp optimizer and binary cross-entropy loss.

    Returns:
        model (Sequential): The compiled CNN model.
    """
    # Initialize a sequential model
    model = Sequential([
        # 1: convolutional layer with 32 filters of size 3x3, ReLU activation, and a max pooling layer with window size 2x2
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        # 2: convolutional layer with 64 filters of size 3x3, ReLU activation, and a max pooling layer with window size 2x2
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        # 3: convolutional layer with 64 filters of size 3x3, ReLU activation, and a max pooling layer with window size 2x2
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        # 4: flatten pixels of resulting image into a 1D vector
        layers.Flatten(),
        # 5: feed result into fully connected dense layer with 64 units and ReLU activation
        layers.Dense(64, activation='relu'),
        # 6. output layer with a single unit and sigmoid activation for binary classification
        layers.Dense(1, activation='sigmoid')
    ])

    # optimizerAdam = Adam(learning_rate=ALPHA) # RMSProp gives much better results
    # Use binary cross-entropy loss function and RMSProp optimizer
    model.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=['accuracy'])

    return model

# Create a new instance of the raw model by loading saved weights
raw_model = build_and_compile_cnn()
raw_model.load_weights('16_epochs.weights.h5')

# Select 10 Reptile and Amphibian images to show in the Gradio interface
def get_sample_image_paths():
    """
    Get sample reptile/amphibian images.

    Returns:
        List[str]: List of image paths.
    """
    # State location of image directory
    dir = "Gradio_Sample_Images"

    # Fetch paths of images
    images = [os.path.join(dir, f) for f in os.listdir(dir) if f.endswith(('.jpg', '.png'))]

    return images

# Create function for Gradio interface
def classify_image_raw_model(inp):
    """
    Classify an input image as either 'Amphibian' or 'Reptile' using the raw CNN Model.

    Args:
        inp (PIL.Image): The input image to classify.

    Returns:
        str: The classification result ('Amphibian' or 'Reptile').
    """
    # Resize and normalize input image
    inp = inp.resize(IMAGE_SIZE)
    inp = np.array(inp) / 255.0
    inp = inp.reshape((-1, 224, 224, 3))

    # Predict class
    prediction = raw_model.predict(inp)
    predicted_class = int(np.round(prediction[0][0]))

    # Map class index to class name
    class_names = {0: 'Amphibian', 1: 'Reptile'}
    return class_names[predicted_class]

def classify_image_ensemble_model(inp):
    """
    Classify an input image as either 'Amphibian' or 'Reptile' using the ensembled pre-trained CNN model.

    Args:
        inp (PIL.Image): The input image to classify.

    Returns:
        str: The classification result ('Amphibian' or 'Reptile').
    """
    # Resize and normalize input image
    inp = inp.resize(IMAGE_SIZE)
    inp = np.array(inp) / 255.0
    inp = inp.reshape((-1, 224, 224, 3)) # For this function, replace any of these to fit the pre-trained model params

    # Predict class
    prediction = raw_model.predict(inp) # For this function, replace this with the ensemble model
    predicted_class = int(np.round(prediction[0][0]))

    # Map class index to class name
    class_names = {0: 'Amphibian', 1: 'Reptile'}
    return class_names[predicted_class]

def classify_images(inp):
    """
    Classify an input image using both models and return both results.

    Args:
        inp (PIL.Image): The input image to classify.

    Returns:
        Tuple[str, str]: The classification results from both models.
    """
    return classify_image_raw_model(inp), classify_image_ensemble_model(inp)

# Create a Gradio interface
interface = gr.Interface(
    fn=classify_images,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(num_top_classes=1, label="Raw Classifier"),
        gr.Label(num_top_classes=1, label="Ensembled Pre-trained Classifier")
    ],
    examples=get_sample_image_paths(),
    title="Amphibian vs Reptile Classifier",
    description="Upload an image to classify it as either an Amphibian or a Reptile."
)
interface.launch(share=True)

# The raw model accurately predicts 9/10 Gradio image examples. It incorrectly classifies one reptile as an amphibian

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://84c8627ac27b61d9eb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
